<a href="https://colab.research.google.com/github/NehaSontakk/PPI-PGM/blob/main/Missing_ko_identification_via_MPP_and_Kofam_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
mpp_output_path_again = 'Ecoli_Metapath.tsv'
mpp_data_again = pd.read_csv(mpp_output_path_again, sep='\t')

row_of_interest_again = mpp_data_again.iloc[0]

pathways_with_1_again = row_of_interest_again[row_of_interest_again == 1].index.tolist()
if 'file' in pathways_with_1_again:
    pathways_with_1_again.remove('file')

pathways_with_1_again


In [12]:
import requests
from bs4 import BeautifulSoup

def scrape_ko_ids_from_definition(module_id):
    url = f"https://www.kegg.jp/module/{module_id}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Attempt to find the table row (`<tr>`) that contains the 'Definition' text
        definition_row = soup.find(lambda tag: tag.name == "td" and "Definition" in tag.text)
        if definition_row:
            # The actual definition with KO IDs should be in the next `<td>` sibling
            definition_data = definition_row.find_next_sibling("td")
            if definition_data:
                # Extract all KO IDs from the links within this `<td>`
                ko_links = definition_data.find_all('a', href=True)
                ko_ids = [link.text for link in ko_links if link.text.startswith('K')]
                return ko_ids

    print("Failed to retrieve or parse module information.")
    return []

# Example usage
module_id = 'M00002'
ko_ids = scrape_ko_ids_from_definition(module_id)
print(ko_ids)


['K01803', 'K00134', 'K00150', 'K00927', 'K11389', 'K01834', 'K15633', 'K15634', 'K15635', 'K01689', 'K27394', 'K00873', 'K12406']


In [13]:
pathway_ko_dict = {}

# Iterate over each pathway and fetch the KO list
for pathway in pathways_with_1_again:
    ko_list = scrape_ko_ids_from_definition(pathway)
    pathway_ko_dict[pathway] = ko_list


In [ ]:
import pandas as pd

# Assuming the KofamScan output file is correctly located and 'pathway_ko_dict' is defined as before
kofamscan_output_df = pd.read_csv('Ecoli_kofamscan_output.tsv', sep='\t')
found_kos = set(kofamscan_output_df['KO'].unique())

def find_missing_kos_per_module(found_kos, expected_kos_per_module):
    missing_kos_per_module = {}
    for module, expected_kos in expected_kos_per_module.items():
        missing_kos = set(expected_kos) - found_kos
        if missing_kos:
            missing_kos_per_module[module] = list(missing_kos)
    return missing_kos_per_module

missing_kos = find_missing_kos_per_module(found_kos, pathway_ko_dict)

# Here, `missing_kos` is the dictionary with each module mapped to its list of missing KOs.
# You can print, analyze, or further manipulate this dictionary as needed.
for module, missing in missing_kos.items():
    print(f"Module {module} is missing KOs: {missing}")


In [36]:
kofamscan_output_df = pd.read_csv('Ecoli_kofamscan_output.tsv', sep='\t')
found_kos = set(kofamscan_output_df['KO'].unique())


def find_missing_kos_per_module(found_kos, expected_kos_per_module):
    missing_kos_per_module = {}
    for module, expected_kos in expected_kos_per_module.items():
        missing_kos = set(expected_kos) - found_kos
        if missing_kos:
            missing_kos_per_module[module] = list(missing_kos)
    return missing_kos_per_module

missing_kos = find_missing_kos_per_module(found_kos, pathway_ko_dict)

for module, missing in missing_kos.items():
    print(f"Module {module} is missing KOs: {missing}")

Module M00002 is missing KOs: ['K11389', 'K15635', 'K12406', 'K00150', 'K27394']
Module M00003 is missing KOs: ['K11532', 'K01596', 'K01622', 'K01086', 'K04041', 'K27394', 'K00150', 'K01623', 'K15635']
Module M00004 is missing KOs: ['K06859', 'K13810', 'K13937', 'K19243', 'K15916', 'K01057']
Module M00006 is missing KOs: ['K13937', 'K19243', 'K01057']
Module M00007 is missing KOs: ['K13810']
Module M00008 is missing KOs: ['K01057']
Module M00009 is missing KOs: ['K00024', 'K00174', 'K00030', 'K00025', 'K18118', 'K00235', 'K18859', 'K00177', 'K00234', 'K00026', 'K01900', 'K05942', 'K00175', 'K25801', 'K01677', 'K01678', 'K00236', 'K18860', 'K01616', 'K00176', 'K00237', 'K01899']
Module M00010 is missing KOs: ['K05942', 'K00030']
Module M00011 is missing KOs: ['K00024', 'K00174', 'K00025', 'K18118', 'K00235', 'K18859', 'K00177', 'K00234', 'K00026', 'K01900', 'K00175', 'K25801', 'K01677', 'K01678', 'K00236', 'K18860', 'K01616', 'K00176', 'K00237', 'K01899']
Module M00015 is missing KOs: [

In [37]:
data_for_df = []
for module, expected_kos in pathway_ko_dict.items():
    missing_ko1 = missing_kos.get(module, [])
    row = {
        'Module': module,
        'Expected_KOs': ", ".join(expected_kos),
        'Missing_KOs': ", ".join(missing_ko1)
    }
    data_for_df.append(row)

pathway_ko_df = pd.DataFrame(data_for_df)

print(pathway_ko_df)

    Module                                       Expected_KOs  \
0   M00002  K01803, K00134, K00150, K00927, K11389, K01834...   
1   M00003  K01596, K01610, K01689, K27394, K01834, K15633...   
2   M00004  K13937, K00036, K19243, K01057, K07404, K00033...   
3   M00006     K13937, K00036, K19243, K01057, K07404, K00033   
4   M00007     K00615, K00616, K13810, K01783, K01807, K01808   
..     ...                                                ...   
74  M00938  K00525, K00526, K10807, K10808, K00940, K18533...   
75  M00939  K09018, K09024, K09020, K09021, K09023, K09019...   
76  M00005                                             K00948   
77  M00120  K00867, K03525, K01947, K09680, K01922, K21977...   
78  M00545  K05708, K05709, K05710, K00529, K05711, K05712...   

                                          Missing_KOs  
0              K11389, K15635, K12406, K00150, K27394  
1   K11532, K01596, K01622, K01086, K04041, K27394...  
2      K06859, K13810, K13937, K19243, K15916, K010

In [38]:
pathway_ko_df

,Module,Expected_KOs,Missing_KOs
0,M00002,"K01803, K00134, K00150, K00927, K11389, K01834...","K11389, K15635, K12406, K00150, K27394"
1,M00003,"K01596, K01610, K01689, K27394, K01834, K15633...","K11532, K01596, K01622, K01086, K04041, K27394..."
2,M00004,"K13937, K00036, K19243, K01057, K07404, K00033...","K06859, K13810, K13937, K19243, K15916, K01057"
3,M00006,"K13937, K00036, K19243, K01057, K07404, K00033","K13937, K19243, K01057"
4,M00007,"K00615, K00616, K13810, K01783, K01807, K01808",K13810
...,...,...,...
74,M00938,"K00525, K00526, K10807, K10808, K00940, K18533...","K18533, K13998, K10807, K10808, K03465"
75,M00939,"K09018, K09024, K09020, K09021, K09023, K09019...",
76,M00005,K00948,
77,M00120,"K00867, K03525, K01947, K09680, K01922, K21977...","K01598, K01947, K03525, K01922, K21977, K02318..."
